<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/transformers_for_SequenceClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install accelerate -U
!pip install datasets
!pip install evaluate
!pip install tensorboard
import torch
import evaluate
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer
from transformers import TrainingArguments
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [ ]:
datasets = load_dataset("lansinuote/ChnSentiCorp")
datasets

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

In [ ]:
datasets = datasets.filter(lambda x:x['label'] is not None)

Filter:   0%|          | 0/9600 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1200 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [ ]:
for data in datasets["train"]:
  print(data)
  break

{'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': 1}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/156M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def process_data(example):
  data = tokenizer(example["text"],padding="max_length",max_length=128,truncation=True,return_tensors="pt")
  data['labels']=example["label"]
  return data
tokenized_datasets=datasets.map(process_data,remove_columns=datasets["train"].column_names,batched=True)

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [ ]:
acc = evaluate.load("accuracy")
f1 = evaluate.load("f1")

In [ ]:
def eval_metric(results):
  preds,labels = results
  preds = preds.argmax(axis=-1)
  acc_score = acc.compute(predictions=preds,references=labels)
  f1_score = f1.compute(predictions=preds,references=labels)
  acc_score.update(f1_score)
  return acc_score

In [ ]:
train_args = TrainingArguments(output_dir="./easy_practice",
                               per_device_train_batch_size=32,
                               per_device_eval_batch_size=128,
                               gradient_accumulation_steps=2,
                               learning_rate=1e-5,
                               weight_decay=0.01,
                               logging_steps=20,
                               evaluation_strategy="epoch",
                               save_strategy="epoch",
                               save_total_limit=5,
                               load_best_model_at_end=True,
                               metric_for_best_model="f1",
                               num_train_epochs=3)

In [ ]:
trainer = Trainer(model=model,
                  args=train_args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["validation"],
                  compute_metrics=eval_metric)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.256800,0.287996,0.893333,0.889465
2,0.234400,0.277744,0.900000,0.899160
3,0.229200,0.274630,0.903333,0.902192


TrainOutput(global_step=450, training_loss=0.22582255840301513, metrics={'train_runtime': 169.0392, 'train_samples_per_second': 170.375, 'train_steps_per_second': 2.662, 'total_flos': 483448106188800.0, 'train_loss': 0.22582255840301513, 'epoch': 3.0})

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.2696964740753174,
 'eval_accuracy': 0.9025,
 'eval_f1': 0.9049553208773355,
 'eval_runtime': 2.2647,
 'eval_samples_per_second': 529.863,
 'eval_steps_per_second': 4.416,
 'epoch': 3.0}

In [ ]:
trainer.predict(tokenized_datasets["test"])

PredictionOutput(predictions=array([[ 0.57252496, -1.2622275 ],
       [ 0.43299907, -0.85860705],
       [ 0.9705529 , -1.6632177 ],
       ...,
       [-1.6609751 ,  2.024191  ],
       [-2.0095396 ,  2.4350796 ],
       [ 2.2109365 , -2.6312149 ]], dtype=float32), label_ids=array([1, 0, 0, ..., 1, 1, 0]), metrics={'test_loss': 0.2696964740753174, 'test_accuracy': 0.9025, 'test_f1': 0.9049553208773355, 'test_runtime': 2.4158, 'test_samples_per_second': 496.721, 'test_steps_per_second': 4.139})

In [ ]:
!tensorboard --logdir "easy_practice/runs"